Shell Slices
==========

**Summary:**    2-D profiles of selected output variables sampled on spherical surfaces with each output $q_{ij}$ defined such that

$q_{ij} = f_i(r_j,\theta,\phi)$

for each specified output $f_i$ and each specified radius $r_j$.

**Subdirectory:**  Shell_Slices

**main_input prefix:** shellslice

**Python Class:** Shell_Slices

**Additional Namelist Variables:**  

* shellslice_levels (indicial) : indices along radial grid at which to output spherical surfaces.

* shellslice_levels_nrm (normalized) : normalized radial grid coordinates at which to output spherical surfaces.


The shell-slice output type allows us to examine how the fluid properties vary on spherical surfaces.

Examining the *main_input* file, we see that the following output values have been denoted for the Shell Slices (see *rayleigh_output_variables.pdf* for mathematical formulae):


| Menu Code  | Description |
|------------|-------------|
| 1          | Radial Velocity |
| 2          | Theta Velocity |
| 3          | Phi Velocity  |




In the example that follows, we demonstrate how to create a 2-D plot of the radial velocity on a Cartesian, lat-lon grid.

Plotting on a lat-lon grid is straightforward and illustrated below.   The shell-slice data structure is also displayed via the help() function in the example below and contains information needed to define the spherical grid for plotting purposes.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

model_type = 2 # 1 for anelastic example, 2 Boussinesq example
base_dir = '/home/nfeatherstone/runs/cig_vis/'
font_size=14     # Font size for plot labels 

if (model_type == 1):
    model_dir = base_dir+'anelastic/'
    # Define some units for plotting purposes
    eunits = r'(erg cm$^{-3}$)'                # energy density
    tunits = '(s)'                             # time
    vunits = r'(cm s$^{-1}$)'                  # velocity
    dunits = '(cm)'                            # distance
    thermal_label = r' Specific Entropy '      # specific entropy
    thermal_units = r'(erg g$^{-1}$ K$^{-1}$)' # specific entropy
    lunits = r'(erg s$^{-1}$)'                 # energy / time (luminosity)
    funits = '(nHz)'                           # Frequency units
    mfunits = '(g cm$^{-2}$ s$^{-1}$)'
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series
    
if (model_type == 2):
    model_dir = base_dir+'Boussinesq/'
    # Define some units for plotting purposes
    eunits = ''                          # energy density
    tunits = '(viscous diffusion times)' # time
    vunits = ''                          # velocity
    dunits = ''                          # distance
    thermal_label = ' Temperature ' 
    thermal_units = '' 
    lunits = ''                          # energy / time (luminosity)
    funits = ''                          # Frequency units
    mfunits = ''
    
    # Next, we set some timestep ranges for plotting purposes
    imin = 0             # minimum iteration number to process for time series
    imax = 10000000      # maximum iteration number to process for time series

In [ ]:
#####################################
#  Shell Slice
from rayleigh_diagnostics import Shell_Slices, build_file_list
import numpy
import matplotlib.pyplot as plt
from matplotlib import ticker, font_manager
# Read the data

files = build_file_list(imin,imax,path=model_dir+'Shell_Slices')
nf = len(files)
ss = Shell_Slices(files[nf-1],path='')

help(ss)

In [ ]:



tindex =1 # Grab the second time index
rindex = 0 # Grab the first radius output
qindex=1 # vphi
sizetuple=(8,8)
plt.rcParams.update({'font.size': font_size})
tsize = 20     # title font size
cbfsize = 10   # colorbar font size

ntheta = ss.ntheta
nphi = ss.nphi
costheta = ss.costheta
theta = numpy.arccos(costheta)


data = ss.vals[:,:,rindex,ss.lut[qindex],tindex]
fig, ax = plt.subplots(figsize=sizetuple)

rms = (numpy.mean(data*data))**0.5

smin = -1.5*rms
smax = 1.5*rms

img = plt.imshow(numpy.transpose(data), extent=[0,360,-90,90], cmap='RdYlBu_r',vmin=smin,vmax=smax)
ax.set_xlabel( 'Longitude (deg)')
ax.set_ylabel( 'Latitude (deg)')
ax.set_title(  'Radial Velocity')

#colorbar ...
cbar = plt.colorbar(img,orientation='horizontal', shrink=0.5, aspect = 15, ax=ax)
cbar.set_label(vunits)
        
tick_locator = ticker.MaxNLocator(nbins=5)
cbar.locator = tick_locator
cbar.update_ticks()
cbar.ax.tick_params(labelsize=cbfsize)   #font size for the ticks

t = cbar.ax.xaxis.label
t.set_fontsize(cbfsize)  # font size for the axis title


plt.tight_layout()
plt.show()

It's also common to plot this sort of data in orthographic projection (i.e., so that it looks like a globe).
In the example shown here, we plot all three velocity components ($u_r$, $u_{\theta}$ and $u_{\phi}$)
projected onto a spherical surface. For demonstration purposes, we illustrate each velocity component using different colormaps, at different latitudinal centers of vantage point etc. (for more details see comments within cell below as well as the Jupyter notebook titled "plot_shells.ipynb").

Note that in order to successfully run the cell below, you also need to have the orthographic projection code "projection.py" within the same directory/folder as this notebook.

In [ ]:
import numpy
import matplotlib.pyplot as plt
from matplotlib import gridspec
from rayleigh_diagnostics import Shell_Slices
from projection import plot_ortho

# This plots various data from a single shell_slice file.
# 3 diferent plots in 1 row and 3 columns are created.
# It's easy to hack this to work with multiple files

s1=Shell_Slices(files[nf-2],path='')
tindex = 0 # use first time slice



data = numpy.zeros((s1.nphi,s1.ntheta),dtype='float64')
costheta = s1.costheta
#nrows=1
#ncols=2
pltin = 9  # Size of each subimage in inches (will be square)


# number of rows and columns
nrow=2
ncol=3

#We use gridspec to set up a grid.  We actually have nrow*2 rows, with every other
#row being a 'spacer' row that's 10% the height of the main rows.
#This setup is one way for the color bars to display nicely.
fig = plt.figure(constrained_layout=False, figsize=(pltin*ncol,pltin*nrow*1.1))
spec = gridspec.GridSpec(ncols=ncol, nrows=nrow*2, figure=fig, height_ratios=[1,.1]*nrow, width_ratios=[1]*ncol)



plt.rcParams.update({'font.size': 16})
#quantities codes to plot -- here all three velocity components
qi =  [1,2, 3]
nm = [r'u$_r$', r'u$_\theta$', r'u$_\phi$']

qinds = [qi, qi]  # Quantity codes to plot
names = [nm, nm]  # Names for labeling

# Shell levels to plot (top row is level 0, bottom row is level 1)
lv1 = [0]*ncol  # top row
lv2 = [1]*ncol  # bottom row
lv = [lv1 , lv2 ]

 # Line style of grid lines
style1=['-','--',':']  # top row
style2=['-', '-', '-'] # bottom row
styles = [style1, style2]  

# width of grid lines for each image (Default: 1)
gwidth1=[0.5 , 1 , 1.5]  # top
gwidth2=[1,1,1]          # bottom
gwidths = [gwidth1, gwidth2] 

# Width of the horizon line or each image (Default: 2)
hwidths1=[2.5,2.5,2.5]  # top
hwidths2=[2,2,2]        #bottom
hwidths=[hwidths1,hwidths2]

# A color table for each image (Default: RdYlBu_r)
cmaps1 = ["RdYlBu_r", "seismic", 'PiYG']  # top row
cmaps2 = ["RdYlBu_r"]*3 # bottom row
cmaps = [cmaps1, cmaps1]

# Plot grids, or not for each image (Default: True)
pgrids1 = [True, True, True] # top row
pgrids2 = [True, True, True] # bottom row
pgrids  = [pgrids1, pgrids2] 

# Latitudinal center of vantage point (Default: 45 N)
latcens1 = [60, 45, 15]  # top
latcens2 = latcens1      #bottom
latcens = [latcens1, latcens2]   

# Longitudinal center of vantage point (Default: 0)
loncens1 = [0,0,0]     # top
loncens2 = loncens1 # bottom 
loncens = [loncens1,loncens2]

##########################################################
# If the grid is plotted, the number of latitude lines
# for the grid can be controlled via the nlats keyword.
# Default: 9
# Note that if nlats is even, the equator will not be drawn
nlats1 = [3,5,7]
nlats2 = [4,6,8]
nlats = [nlats1, nlats1]

##############################################################################
# Similarly, the nlons keyword can be used to control longitude lines
# More precisely, it controls the number of MERIDIANS (great circles) drawn
# Default:  8
nlons1 = [4,8,12]
nlons2 = [4,12,16]
nlons = [nlons1,nlons1]

#Longitude grid-lines can be drawn in one of two ways:
# 1)  Completely to the pole (polar_style = 'polar')
# 2)  Truncated at the last line of latitue drawn (polar_style = 'truncated')
# Default:  "truncated"
pstyle1 = ['truncated', 'polar', 'truncated']
pstyle = [pstyle1, pstyle1]


##############################################################
# We can also control the way in which the image is saturated
# via the scale_type keyword.   There are three possibilities:
# 1) scale_type=['rms', a], where a is of type 'float'
#    In this instance, the image bounds are -a*rms(data), +a*rms(data)
# 2) scale_type = ['abs', a]
#    In this instance, the image bounds are -a*abs(data), +a*abs(data)
# 3) scale_type= ['force', [a,b]]
#    In this instance, the image bounds are a,b
# 4) scale_type = [None,None]
#    In this instance, the image bounds are min(image), max(image)
# Default:  [None,None]
# Note that rms and abs are taken over projected image values, not input data
# (you only see half the data in the image)

scale_type1 = [['rms',2.0 ], [None,None], ['abs', 0.5]]
scale_type2 = [['force', [-1500,1500]], ['force',[-10000,10000]], ['rms',2.5]]
scale_types = [scale_type1, scale_type1]

# Number of pixels across each projected, interpolated image
# 768 is the default and seems to do a reasonable job
nyzi = 768



for j in range(ncol):
    for i in range(nrow):
        data[:,:] = s1.vals[:,:,lv[i][j],s1.lut[qinds[i][j]],tindex]    

        row_ind = 2*i # skip over space allowed for color bars
        col_ind = j

        print("Plotting row,column: ", row_ind//2, col_ind)
        ax    = fig.add_subplot(spec[row_ind,col_ind])
        cspec = spec[row_ind+1,col_ind]
        caxis=None

        plot_ortho(data,s1.costheta,fig,ax,caxis, hwidth=hwidths[i][j], gridstyle=styles[i][j], 
                   gridwidth=gwidths[i][j], nyz=nyzi, colormap=cmaps[i][j], 
                   plot_grid=pgrids[i][j], latcen=latcens[i][j], loncen= loncens[i][j],
                   pole_style=pstyle[i][j], nlats = nlats[i][j],scale_type=scale_types[i][j])
        ptitle=names[i][j]+"   (r_index = "+str(lv[i][j])+")"
        ax.set_title(ptitle)


# You can save the plot as a figure
#plt.savefig('flows.pdf')